# Summary

Experiment with ways to support passing a list of prompts to query method. Some backends don't support this natively, others do, but none automatically would return the format I want.

In [196]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [197]:
from collections import defaultdict, deque
import matplotlib.pyplot as plt
import numpy as np
import openai
import os
import pandas as pd
from pathlib import Path
from threading import Thread

from jabberwocky.config import C
from jabberwocky.openai_utils import load_prompt, load_openai_api_key, \
    GPTBackend
from htools import *

In [3]:
cd_root()

Current directory: /Users/hmamin/jabberwocky


## Option 1: make thread that returns value so we can run a separate query for each thread

In [107]:
class ReturningThread(Thread):

    @add_docstring(Thread)
    def __init__(self, group=None, target=None, name=None,
                 args=(), kwargs=None, *, daemon=None):
        """This is identical to a regular thread except that the join method
        returns the value returned by your target function. The
        Thread.__init__ docstring is shown below for the sake of convenience.
        """
        super().__init__(group=group, target=target, name=name,
                         args=args, kwargs=kwargs, daemon=daemon)
        self.result = None

    def run(self):
        self.result = self._target(*self._args, **self._kwargs)
        
    def join(self, timeout=None):
        super().join(timeout)
        return self.result

In [79]:
def foo(x, wait=2):
    time.sleep(wait)
    return x

In [99]:
def foo_inv(x, wait=2):
    time.sleep(1 / wait)
    return x

In [104]:
def foo_random(x, max_wait=5):
    wait = np.random.uniform(low=0, high=max_wait)
    print(wait, flush=True)
    time.sleep(wait)
    return x

In [14]:
# Returns values but is slow (sync execution).
res = [foo(i) for i in range(5)]
res

[0, 1, 2, 3, 4]

In [12]:
threads = [Thread(target=foo, args=(i,)) for i in range(5)]
for thread in threads:
    thread.start()

# Regular thread returns None.
res = [thread.join() for thread in threads]
res

[None, None, None, None, None]

In [18]:
threads = [ReturningThread(target=foo, args=(i,)) for i in range(5)]
for thread in threads:
    thread.start()

# ReturningThread returns values!
res = [thread.join() for thread in threads]
res

[0, 1, 2, 3, 4]

In [98]:
threads = [ReturningThread(target=foo_inv, args=(i, i)) for i in range(1, 5)]
for thread in threads:
    thread.start()

# ReturningThread returns values!
res = [thread.join() for thread in threads]
res

wait 1.0
wait 0.5
wait 0.3333333333333333
wait 0.25


[1, 2, 3, 4]

In [108]:
threads = [ReturningThread(target=foo_random, args=(i, 5))
           for i in range(1, 5)]
for thread in threads:
    thread.start()

# ReturningThread returns values!
res = [thread.join() for thread in threads]
res

3.7145703512021404
0.5710566753268154
2.778421106481786
0.44917966624138606


[1, 2, 3, 4]

## Try integrating into GPTBackend

In [8]:
# TODO: no guarantees these threads return in the right order, though, right?

In [156]:
gpt = GPTBackend()
gpt.ls()

Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_stream_response.pkl.

Base: https://api.openai.com
Query func: <function query_gpt_banana at 0x11e51f2f0>


In [157]:
gpt.switch('huggingface')
gpt.ls()

Switching openai backend to "huggingface".

Base: https://api.openai.com
Query func: <function query_gpt_huggingface at 0x1100820d0>


In [8]:
prompts = [
    'Six million years after the pandemic,',
    'The stegosaurus'
]
kwargs = {'max_tokens': 10}

In [28]:
threads = [ReturningThread(target=gpt.query, args=(prompt,), kwargs=kwargs) 
           for prompt in prompts]
for thread in threads:
    thread.start()
res = [thread.join() for thread in threads]

In [29]:
res

[('the world is still in the grip of a global',
  {'generated_text': ' the world is still in the grip of a global'}),
 ('is a large, large, and highly intelligent animal',
  {'generated_text': ' is a large, large, and highly intelligent animal'})]

In [32]:
threads = [ReturningThread(target=gpt.query,
                           args=(prompt,), 
                           kwargs={**kwargs, 'n': 3, 
                                   'logprobs': 4, 'engine_i': 1}) 
           for prompt in prompts]
for thread in threads:
    thread.start()
res = [thread.join() for thread in threads]

In [34]:
lmap(len, *res)

[2, 2]

In [36]:
res[0][0]

['the population of New York City is poised to rise',
 'the pandemic strain of influenza spreads and mutates',
 'our species is still struggling to deal with the effects']

In [37]:
res[0][1]

[{'generated_text': ' the population of New York City is poised to rise'},
 {'generated_text': ' the pandemic strain of influenza spreads and mutates'},
 {'generated_text': ' our species is still struggling to deal with the effects'}]

In [38]:
res[1][0]

['is one of the more remarkable prehistoric dinosaurs, and',
 ', or giant pterosaur from the late',
 'fossil, or dinosaur\nFossil bones of']

In [39]:
res[1][1]

[{'generated_text': ' is one of the more remarkable prehistoric dinosaurs, and'},
 {'generated_text': ', or giant pterosaur from the late'},
 {'generated_text': ' fossil, or dinosaur\nFossil bones of'}]

In [158]:
gpt.switch('gooseai')
gpt.ls()

Switching openai backend to "gooseai".

Base: https://api.goose.ai/v1
Query func: <function query_gpt3 at 0x1100829d8>


In [41]:
threads = [ReturningThread(target=gpt.query,
                           args=(prompt,), 
                           kwargs={'max_tokens': 8, 'n': 2, 
                                   'logprobs': 5, 'engine_i': 0}) 
           for prompt in prompts]
for thread in threads:
    thread.start()
res = [thread.join() for thread in threads]

In [50]:
len(res)

2

In [43]:
res[0][0]

['the world is still in the grip of',
 'scientists still do not know whether humans are']

In [44]:
res[1][0]

['is a fossilized dinosaur named by the',
 'is a famous carnivorous dinosaur from the']

In [52]:
len(res[0])#[1][1]

2

In [75]:
texts, resps = list(zip(*res))
texts

(['the world is still in the grip of',
  'scientists still do not know whether humans are'],
 ['is a fossilized dinosaur named by the',
  'is a famous carnivorous dinosaur from the'])

In [80]:
# resps[i][j] corresponds to prompt i, completion j.
[completion['logprobs'].tokens for completion in resps[0]]

[[' the', ' world', ' is', ' still', ' in', ' the', ' grip', ' of'],
 [' scientists',
  ' still',
  ' do',
  ' not',
  ' know',
  ' whether',
  ' humans',
  ' are']]

In [81]:
[completion['logprobs'].tokens for completion in resps[1]]

[[' is', ' a', ' fossil', 'ized', ' dinosaur', ' named', ' by', ' the'],
 [' is', ' a', ' famous', ' carniv', 'orous', ' dinosaur', ' from', ' the']]

In [159]:
gpt.ls()


Base: https://api.goose.ai/v1
Query func: <function query_gpt3 at 0x1100829d8>


In [88]:
threads2 = [ReturningThread(target=gpt.query,
                           args=(prompt,), 
                           kwargs={'max_tokens': 8, 'n': 1, 
                                   'logprobs': 5, 'engine_i': 0}) 
            for prompt in prompts]
for thread in threads2:
    thread.start()
res2 = [thread.join() for thread in threads2]

In [92]:
texts2, resps2 = list(zip(*res2))

In [93]:
texts2

('the future of the world’s', 'The stegosaurus (Ste')

In [95]:
# Because only 1 completion per prompt, resps is a dict instead of a list of 
# dicts.
resps2[0]

{'finish_reason': 'length',
 'index': 0,
 'logprobs': <OpenAIObject at 0x125e93d58> JSON: {
   "text_offset": [
     0,
     4,
     11,
     14,
     18,
     24,
     24,
     25
   ],
   "token_logprobs": [
     -1.7412109375,
     -5.03515625,
     -0.361083984375,
     -1.7802734375,
     -1.8515625,
     -1.89453125,
     -0.0006651878356933594,
     -0.00013065338134765625
   ],
   "tokens": [
     " the",
     " future",
     " of",
     " the",
     " world",
     "\ufffd",
     "\ufffd",
     "s"
   ],
   "top_logprobs": [
     {
       " a": -2.90234375,
       " it": -3.947265625,
       " scientists": -4.09375,
       " the": -1.7412109375,
       " we": -2.748046875
     },
     {
       " city": -4.51171875,
       " human": -4.06640625,
       " pand": -4.2109375,
       " virus": -2.9921875,
       " world": -1.6845703125
     },
     {
       " is": -2.072265625,
       " looks": -3.4296875,
       " of": -0.361083984375,
       " remains": -3.44921875,
       " still

In [9]:
with gpt('huggingface'):
    hf_res = gpt.query('I want', engine_i=1, max_tokens=5, n=2)

Switching openai backend to "huggingface".
Switching  backend back to "huggingface".


In [10]:
Results(text=hf_res[0], full=hf_res[1])

Results(text=['to give you one big', 'to show you some pictures'], full=[{'generated_text': ' to give you one big'}, {'generated_text': ' to show you some pictures'}])

In [ ]:
# Better interface?
# texts, full_resps = gpt.query([p1, p2, p3], n=2)

## Test streaming mode

Need a better understanding of what using streaming mode is like before I decide about streaming interface for np or nc > 1.

In [11]:
from base64 import b64encode

from jabberwocky.openai_utils import query_gpt3, query_gpt_huggingface, \
    query_gpt_banana, query_gpt_j, query_gpt_repeat

In [12]:
# Was toying with idea of adding this to gpt.query warnings to make the 
# messages unique, in the hope that this would ensure they're always shown
# rather than just once. But a. I'm not sure if that's how they define 
# duplicates, and b. I'm seeing code defined in nb seems to always show 
# warnings, not just once, so I'm not sure what to make of that. Still 
# eventually want to write a func like this (maybe moreso for creating new
# file paths when encountering collisions) but that should have a more limited
# set of possible characters.
def random_str(length, lower=True):
    rand = b64encode(os.urandom(length)).decode()[:length]
    return rand.lower() if lower else rand

In [13]:
for i in range(15):
    rand = random_str(i)
    print(i, rand)

0 
1 d
2 1g
3 p4u
4 dpvk
5 ijbgo
6 a1hztl
7 7qzb4lz
8 cxcnot6l
9 jthktr7ne
10 lueazbofkx
11 brpiuhixuva
12 /j6h2vyqih8m
13 rnv/a+azvmlsr
14 row/b8sipg+zfr


In [14]:
os.urandom(10)

b' \x97\xc1\xc3\x0eB29\x9b\xf7'

In [72]:
with gpt('repeat'):
    repeat_res = gpt.query('I want to go to there.',
                           max_tokens=5, stream=True)

for txt_, full_ in repeat_res:
    print(txt_, full_)

Switching openai backend to "repeat".
Switching  backend back to "huggingface".
I  {'index': 0, 'finish_reason': None}
want  {'index': 0, 'finish_reason': None}
to  {'index': 0, 'finish_reason': None}
go  {'index': 0, 'finish_reason': None}
to  {'index': 0, 'finish_reason': None}
there.  {'index': 0, 'finish_reason': 'dummy'}



In [71]:
with gpt('repeat'):
    repeat_res = gpt.query('I want to go to there.',
                           max_tokens=5, stream=True, n=2)

for txt_, full_ in repeat_res:
    print(txt_, full_)
    if full_['finish_reason']:
        print()

Switching openai backend to "repeat".
Switching  backend back to "huggingface".
I  {'index': 0, 'finish_reason': None}
want  {'index': 0, 'finish_reason': None}
to  {'index': 0, 'finish_reason': None}
go  {'index': 0, 'finish_reason': None}
to  {'index': 0, 'finish_reason': None}
there.  {'index': 0, 'finish_reason': 'dummy'}

I  {'index': 1, 'finish_reason': None}
want  {'index': 1, 'finish_reason': None}
to  {'index': 1, 'finish_reason': None}
go  {'index': 1, 'finish_reason': None}
to  {'index': 1, 'finish_reason': None}
there.  {'index': 1, 'finish_reason': 'dummy'}



In [73]:
with gpt('banana'):
    for txt_, full_ in gpt.query('Who are you?',
                                 max_tokens=5, stream=True):
        print(txt_, full_)
        if full_['finish_reason']:
            print()

Switching openai backend to "banana".


I'm  {'id': 'aa2f677a-bc98-4e76-b3e7-ab8d6e75302f', 'message': 'success', 'created': 1649128641, 'apiVersion': '26 Nov 2021', 'modelOutputs': [{'output': "\n\nI'm a", 'input': 'Who are you?'}], 'index': 0, 'finish_reason': None}
a  {'id': 'aa2f677a-bc98-4e76-b3e7-ab8d6e75302f', 'message': 'success', 'created': 1649128641, 'apiVersion': '26 Nov 2021', 'modelOutputs': [{'output': "\n\nI'm a", 'input': 'Who are you?'}], 'index': 0, 'finish_reason': 'dummy'}

Switching  backend back to "huggingface".


In [74]:
with gpt('huggingface'):
    for txt_, full_ in gpt.query('Who are you?',
                                 max_tokens=5, stream=True):
        print(txt_, full_)

Switching openai backend to "huggingface".


/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:278: UserWarning: query_gpt_huggingface received unused kwargs {'stream': True}.
  warnings.warn('query_gpt_huggingface received unused kwargs '




I  {'generated_text': '\n\nI am a', 'index': 0, 'finish_reason': None}
am  {'generated_text': '\n\nI am a', 'index': 0, 'finish_reason': None}
a  {'generated_text': '\n\nI am a', 'index': 0, 'finish_reason': 'dummy'}
Switching  backend back to "huggingface".


In [77]:
with gpt('huggingface'):
    for txt_, full_ in gpt.query('Who are you?',
                                 max_tokens=5, stream=True, n=2, engine_i=1):
        print(txt_, full_)
        if full_['finish_reason']:
            print()

Switching openai backend to "huggingface".


I'm  {'generated_text': "\n\nI'm going", 'index': 0, 'finish_reason': None}
going  {'generated_text': "\n\nI'm going", 'index': 0, 'finish_reason': 'dummy'}

  {'generated_text': ' Are you being honest or', 'index': 1, 'finish_reason': None}
Are  {'generated_text': ' Are you being honest or', 'index': 1, 'finish_reason': None}
you  {'generated_text': ' Are you being honest or', 'index': 1, 'finish_reason': None}
being  {'generated_text': ' Are you being honest or', 'index': 1, 'finish_reason': None}
honest  {'generated_text': ' Are you being honest or', 'index': 1, 'finish_reason': None}
or  {'generated_text': ' Are you being honest or', 'index': 1, 'finish_reason': 'dummy'}

Switching  backend back to "huggingface".


In [141]:
[row.choices[0].keys() for row in load(C.mock_stream_paths[True])]

Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_stream_response.pkl.


[dict_keys(['text', 'index', 'logprobs', 'finish_reason']),
 dict_keys(['text', 'index', 'logprobs', 'finish_reason']),
 dict_keys(['text', 'index', 'logprobs', 'finish_reason']),
 dict_keys(['text', 'index', 'logprobs', 'finish_reason']),
 dict_keys(['text', 'index', 'logprobs', 'finish_reason'])]

## Experimenting with streaming text AND dict

In [17]:
from itertools import cycle

In [18]:
def stream_words(text):
    """Like stream_chars but splits on spaces. Realized stream_chars was a bad
    idea because we risk giving SPEAKER turns like
    "This is over. W" and "hat are you doing next?", neither of which would be
    pronounced as intended. We yield with a space for consistency with the
    other streaming interfaces which require no further postprocessing.
    """
    for word in text.split(' '):
        yield word + ' '

In [19]:
def stream_response(text:str, full:dict):
    yield from zip(stream_words(text), cycle([full]))

In [33]:
def containerize(*args):
    res = []
    for arg in args:
        if listlike(arg):
            res.append(arg)
        else:
            res.append([arg])
    return res

In [34]:
# Note: this is probably massively over-engineered for mock streaming, but 
# I'll need to do something like this if I want to support real streaming 
# where nc and/or np > 1 so it was probably useful to work through this logic
# anyway.
def stream_multi_response(texts:list, fulls:list):
    texts, fulls = containerize(texts, fulls)
    for i, (text, full) in enumerate(zip(texts, fulls)):
        queue = deque()
        gen = stream_response(text, 
                              {**full, 'index': i, 'finish_reason': None})
        done = False
        # Yield items while checking if we're at the last item so we can mark
        # it with a finish_reason. This lets us know when one completion ends.
        while True:
            try:
                tok, tok_full = next(gen)
                queue.append((tok, tok_full))
            except StopIteration:
                done = True
            
            while len(queue) > 1:
                tok, tok_full = queue.popleft()
                yield tok, tok_full
            if done: break
        tok, tok_full = queue.popleft()
        tok_full['finish_reason'] = 'dummy'    
        yield tok, tok_full

In [35]:
containerize('abc', {'text': 'def'})

[['abc'], [{'text': 'def'}]]

In [36]:
containerize(['abc'], [{'text': 'def'}])

[['abc'], [{'text': 'def'}]]

In [37]:
containerize(['abc', 'hij'], [{'text': 'def'}, {'text': 'aka'}])

[['abc', 'hij'], [{'text': 'def'}, {'text': 'aka'}]]

In [38]:
txt = 'Santa is coming to town.'
for tok in stream_words(txt):
    print(repr(tok))

'Santa '
'is '
'coming '
'to '
'town. '


In [39]:
for tok, full in stream_response(txt, {}):
    print(repr(tok), full)

'Santa ' {}
'is ' {}
'coming ' {}
'to ' {}
'town. ' {}


In [89]:
# np > 1
with gpt('huggingface'):
    hf_res = gpt.query('I want', engine_i=1, max_tokens=5, n=2)

Switching openai backend to "huggingface".
Switching  backend back to "huggingface".


In [40]:
hf_res

(['to give you one big', 'to show you some pictures'],
 [{'generated_text': ' to give you one big'},
  {'generated_text': ' to show you some pictures'}])

In [41]:
for tok, full in stream_multi_response(*hf_res):
    print('>>> ', tok, full)

>>>  to  {'generated_text': ' to give you one big', 'index': 0, 'finish_reason': None}
>>>  give  {'generated_text': ' to give you one big', 'index': 0, 'finish_reason': None}
>>>  you  {'generated_text': ' to give you one big', 'index': 0, 'finish_reason': None}
>>>  one  {'generated_text': ' to give you one big', 'index': 0, 'finish_reason': None}
>>>  big  {'generated_text': ' to give you one big', 'index': 0, 'finish_reason': 'dummy'}
>>>  to  {'generated_text': ' to show you some pictures', 'index': 1, 'finish_reason': None}
>>>  show  {'generated_text': ' to show you some pictures', 'index': 1, 'finish_reason': None}
>>>  you  {'generated_text': ' to show you some pictures', 'index': 1, 'finish_reason': None}
>>>  some  {'generated_text': ' to show you some pictures', 'index': 1, 'finish_reason': None}
>>>  pictures  {'generated_text': ' to show you some pictures', 'index': 1, 'finish_reason': 'dummy'}


In [42]:
# nc = 1, already containerized.
for tok, full in stream_multi_response(['so'], [{'response': 'so'}]):
    print('>>> ', tok, full)

>>>  so  {'response': 'so', 'index': 0, 'finish_reason': 'dummy'}


In [44]:
# nc = 1, not yet containerized.
for tok, full in stream_multi_response('so', {'response': 'so'}):
    print('>>> ', tok, full)

>>>  so  {'response': 'so', 'index': 0, 'finish_reason': 'dummy'}


In [43]:
# Empty response.
for tok, full in stream_multi_response([], []):
    print('>>> ', tok, full)

In [100]:
# np > 1, stream=True
with gpt('gooseai'):
    goose_res = openai.Completion.create(
        prompt=txt,
        engine=GPTBackend.engine(0),
        max_tokens=5,
        logprobs=3,
        n=2,
        stream=True
    )

Switching openai backend to "gooseai".
Switching  backend back to "huggingface".


In [116]:
# np > 1, stream=True
with gpt('openai'):
    open_res = openai.Completion.create(
        prompt=txt,
        engine=GPTBackend.engine(0),
        max_tokens=5,
        logprobs=3,
        n=2,
        stream=True
    )

Switching openai backend to "openai".
Switching  backend back to "huggingface".


In [102]:
_goose_res = []
for obj in goose_res:
    print(obj)
    _goose_res.append(obj)
    print(spacer())

{
  "choices": [
    {
      "finish_reason": null,
      "index": 0,
      "logprobs": {
        "text_offset": [
          0
        ],
        "token_logprobs": [
          -3.80078125
        ],
        "tokens": [
          " In"
        ],
        "top_logprobs": [
          {
            " And": -2.826171875,
            " The": -3.03125,
            "bytes:'\\n'": -1.30859375
          }
        ]
      },
      "text": " In",
      "token_index": 0
    }
  ],
  "created": 1649026112,
  "id": "01184be1-0c83-4906-a88c-9b71165a0da0",
  "model": "gpt-neo-2-7b",
  "object": "text_completion"
}

-------------------------------------------------------------------------------

{
  "choices": [
    {
      "finish_reason": null,
      "index": 0,
      "logprobs": {
        "text_offset": [
          3
        ],
        "token_logprobs": [
          -5.8125
        ],
        "tokens": [
          " mid"
        ],
        "top_logprobs": [
          {
            " a": -2.478515625,


In [117]:
_open_res = []
for obj in open_res:
    print(obj)
    _open_res.append(obj)
    print(spacer())

{
  "choices": [
    {
      "finish_reason": null,
      "index": 0,
      "logprobs": {
        "text_offset": [
          24
        ],
        "token_logprobs": [
          -8.621929
        ],
        "tokens": [
          " Feeling"
        ],
        "top_logprobs": [
          {
            "\n": -2.4186804,
            " I": -2.6238666,
            " She": -2.4244554
          }
        ]
      },
      "text": " Feeling"
    }
  ],
  "created": 1649026406,
  "id": "cmpl-4t3O2F9Xf3GFJFEVA8q86pCQpTCaD",
  "model": "ada:2020-05-03",
  "object": "text_completion"
}

-------------------------------------------------------------------------------

{
  "choices": [
    {
      "finish_reason": null,
      "index": 0,
      "logprobs": {
        "text_offset": [
          32
        ],
        "token_logprobs": [
          -5.186215
        ],
        "tokens": [
          " her"
        ],
        "top_logprobs": [
          {
            " a": -2.1202018,
            " like": -2.64

In [118]:
len(_goose_res), len(_open_res)

(10, 10)

In [115]:
[(row.choices[0].logprobs.tokens, row.choices[0].finish_reason) 
 for row in _goose_res]

[([' In'], None),
 ([' mid'], None),
 (['-'], None),
 (['January'], None),
 ([','], 'length'),
 ([' There'], None),
 ([' is'], None),
 ([' not'], None),
 ([' a'], None),
 ([' Santa'], 'length')]

In [119]:
[(row.choices[0].logprobs.tokens, row.choices[0].finish_reason) 
 for row in _open_res]

[([' Feeling'], None),
 ([' her'], None),
 ([' I'], None),
 (["'m"], None),
 ([' sure'], None),
 ([' presence'], None),
 ([' you'], None),
 ([','], None),
 ([' Des'], 'length'),
 ([' can'], 'length')]

In [120]:
[row.choices[0].text for row in _open_res]

[' Feeling',
 ' her',
 ' I',
 "'m",
 ' sure',
 ' presence',
 ' you',
 ',',
 ' Des',
 ' can']

In [151]:
# Thought we might be able to use id to reconstruct each completion but that
# doesn't work.
[row['id'] for row in _open_res]

['cmpl-4t3O2F9Xf3GFJFEVA8q86pCQpTCaD',
 'cmpl-4t3O2F9Xf3GFJFEVA8q86pCQpTCaD',
 'cmpl-4t3O2F9Xf3GFJFEVA8q86pCQpTCaD',
 'cmpl-4t3O2F9Xf3GFJFEVA8q86pCQpTCaD',
 'cmpl-4t3O2F9Xf3GFJFEVA8q86pCQpTCaD',
 'cmpl-4t3O2F9Xf3GFJFEVA8q86pCQpTCaD',
 'cmpl-4t3O2F9Xf3GFJFEVA8q86pCQpTCaD',
 'cmpl-4t3O2F9Xf3GFJFEVA8q86pCQpTCaD',
 'cmpl-4t3O2F9Xf3GFJFEVA8q86pCQpTCaD',
 'cmpl-4t3O2F9Xf3GFJFEVA8q86pCQpTCaD']

In [159]:
# index points to which completion each new token belongs to.
completions = defaultdict(list)
for row in _open_res:
    completions[row['choices'][0]['index']].append(row['choices'][0].text)

In [160]:
completions

defaultdict(list,
            {0: [' Feeling', ' her', ' presence', ',', ' Des'],
             1: [' I', "'m", ' sure', ' you', ' can']})

In [163]:
# index points to which completion each new token belongs to.
# completions = defaultdict(list)
for row in _goose_res:
    print(row['choices'][0]['index'], row['choices'][0]['finish_reason'])

0 None
0 None
0 None
0 None
0 length
1 None
1 None
1 None
1 None
1 length


In [164]:
# index points to which completion each new token belongs to.
# completions = defaultdict(list)
for row in _open_res:
    print(row['choices'][0]['index'], row['choices'][0]['finish_reason'])

0 None
0 None
1 None
1 None
1 None
0 None
1 None
0 None
0 length
1 length


In [165]:
with gpt('repeat'):
    print('stream=False\n', gpt.query(txt))
    print('\nstream=True')
    for tok, full in gpt.query(txt, stream=True):
        print(repr(tok), full)

Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_stream_response.pkl.
Switching openai backend to "repeat".
stream=False
 ('Santa is coming to town.', {})

stream=True
'Santa ' {}
'is ' {}
'coming ' {}
'to ' {}
'town. ' {}
Switching  backend back to "huggingface".


/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:923: UserWarning: strip_output=True is not supported in stream mode. Automatically setting it to False.
  warnings.warn('strip_output=True is not supported in stream '
/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:928: UserWarning: Streaming mode does not support manual truncation of stop phrases and your current backend has limited support for truncation.
  'Streaming mode does not support manual truncation of '
/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:391: UserWarning: Unused kwargs {'stream': True} received by query_gpt_repeat.
  warnings.warn(f'Unused kwargs {kwargs} received by query_gpt_repeat.')


In [166]:
with gpt('repeat'):
    print('stream=False\n', gpt.query(txt, n=3))
#     print('\nstream=True')
#     for tok, full in gpt.query(txt, stream=True):
#         print(repr(tok), full)

Switching openai backend to "repeat".
stream=False
 (['Santa is coming to town.', 'Santa is coming to town.', 'Santa is coming to town.'], [{}, {}, {}])
Switching  backend back to "huggingface".


In [145]:
with gpt('huggingface'):
    tmp = gpt.query(txt, max_tokens=5)
    print('stream=False\n', tmp)
    print('\nstream=True')
    for tok, full in gpt.query(txt, stream=True, max_tokens=5):
        print(repr(tok), full)

Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_stream_response.pkl.
Switching openai backend to "huggingface".
stream=False
 ('The city is', {'generated_text': '\n\nThe city is'})

stream=True


/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:923: UserWarning: strip_output=True is not supported in stream mode. Automatically setting it to False.
  warnings.warn('strip_output=True is not supported in stream '
/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:928: UserWarning: Streaming mode does not support manual truncation of stop phrases and your current backend has limited support for truncation.
  'Streaming mode does not support manual truncation of '
/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:275: UserWarning: query_gpt_huggingface received unused kwargs {'stream': True}.
  warnings.warn('query_gpt_huggingface received unused kwargs '


'\n\nThe ' {'generated_text': '\n\nThe city is'}
'city ' {'generated_text': '\n\nThe city is'}
'is ' {'generated_text': '\n\nThe city is'}
Switching  backend back to "huggingface".


In [142]:
for tok, full in stream_response(tmp[0], tmp[1]):
    print(full, tok)

{'generated_text': '\n\nThe city is'} The 
{'generated_text': '\n\nThe city is'} city 
{'generated_text': '\n\nThe city is'} is 


In [139]:
for row in stream_response(*tmp):
    print(row)

('The ', {'generated_text': '\n\nThe city is'})
('city ', {'generated_text': '\n\nThe city is'})
('is ', {'generated_text': '\n\nThe city is'})


In [171]:
hf_res

Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_stream_response.pkl.


(['to give you one big', 'to show you some pictures'],
 [{'generated_text': ' to give you one big'},
  {'generated_text': ' to show you some pictures'}])

In [170]:
for row in stream_response(*hf_res):
    print(row)

AttributeError: 'list' object has no attribute 'split'

## Try using ReturningThread to prototype handling multiple prompts

In [344]:
from datetime import datetime
import logging
from threading import Lock

from jabberwocky.utils import with_signature, JsonlinesLogger, load_api_key,\
    strip, squeeze, JsonlinesFormatter, touch
from jabberwocky.openai_utils import truncate_at_first_stop

In [351]:
class GPTBackend:
    """
    Examples
    --------
    gpt = GPTBackend()

    # Default backend is openai.
    openai_res = gpt.query(**kwargs)

    with gpt('gooseai'):
        # Now we're using the gooseai backend.
        gooseai_res = gpt.query(**kwargs)

    # Now we're back to using openai.
    openai_res_2 = gpt.query(**kwargs)

    # Now we'll switch to gooseai and changes will persist since we're not
    # using a context manager.
    gpt.switch('gooseai')
    gooseai_res_2 = gpt.query(**kwargs)
    """

    logger = JsonlinesLogger(
        f'./data/logs/{datetime.today().strftime("%Y.%m.%d")}.jsonlines'
    )
    lock = Lock()

    # Only include backends here that actually should change the
    # openai.api_base value (these will probably be backends that require no
    # or minimal mock_funcs).
    name2base = {
        'openai': 'https://api.openai.com',
        'gooseai': 'https://api.goose.ai/v1',
    }

    # Order matters: keep openai first so name2key initialization works.
    name2func = {
        'openai': query_gpt3,
        'gooseai': query_gpt3,
        'huggingface': query_gpt_huggingface,
        'hobby': query_gpt_j,
        'repeat': query_gpt_repeat,
        'banana': query_gpt_banana
    }

    # Names of backends that perform stop word truncation how we want (i.e.
    # allow us to specify stop phrases AND truncate before the phrase rather
    # than after, if we encounter one).
    skip_trunc = {'openai'}

    name2key = {}
    for name in name2func:
        if name in {'hobby', 'repeat'}:
            name2key[name] = f'<{name.upper()} BACKEND: FAKE API KEY>'
        else:
            name2key[name] = load_api_key(name)

    def __init__(self):
        self.new_name = ''
        self.old_name = ''
        self.old_key = ''

    def __call__(self, name):
        """__enter__ can't take arguments so we need to specify this here.
        Notice that name is auto-lowercased and spaces are removed.
        """
        new_name = name.lower().replace(' ', '')
        if new_name not in self.name2func:
            raise ValueError(f'Invalid name {name}. Valid options are: '
                             f'{list(self.name2func)}')

        self.new_name = new_name
        self.old_name = self.current()
        return self

    def __enter__(self):
        """Change backend to the one specified in __call__, which is
        automatically called first when using `with` syntax.
        """
        print(f'Switching openai backend to "{self.new_name}".')
        # Store an attribute on openai itself to reduce risk of bugs caused by
        # GPTBackend being deleted or recreated. Previously used a
        # self.base2name mapping to retrieve the current name but that doesn't
        # work when multiple names use the same base (e.g. huggingface and
        # hobby API backends can't be identified just by their base with
        # this implementation).
        openai.curr_name = self.new_name
        self.old_key, openai.api_key = openai.api_key, \
            self.name2key[self.new_name]
        if self.new_name in self.name2base:
            openai.api_base = self.name2base[self.new_name]

    def __exit__(self, exc_type, exc_val, traceback):
        """Revert to previously used backend on contextmanager exit."""
        print(f'Switching  backend back to "{self.old_name}".')
        openai.api_key = self.old_key
        if self.old_name in self.name2base:
            openai.api_base = self.name2base[self.old_name]
        openai.curr_name = self.old_name
        self.clear()

    @classmethod
    def ls(cls):
        """Print current state of the backend: api_base, api_key, and 
        mock_func. Mostly useful for debugging and sanity checks.
        """
        print('\nBase:', openai.api_base)
        print('Query func:', cls._get_query_func())

    @classmethod
    def backends(cls):
        """List all valid backend names. We could always access these via a
        class attribute but this name is easier to remember.

        Returns
        -------
        list[str]
        """
        return list(cls.name2func)

    def clear(self):
        """Reset instance variables tracking that were used to restore
        previous backend.
        """
        self.old_key = self.old_name = self.new_name = ''

    def switch(self, name):
        """Switch backend and make changes persist, unlike in context manager
        where we reset them on exit.

        Parameters
        ----------
        name: str
            One of (openai, gooseai).
        """
        self(name=name).__enter__()
        self.clear()

    @staticmethod
    def current():
        """Get current backend name, e.g. "gooseai". If we've ever switched
        backend with GPTBackend, openai.curr_name
        should exist. If not, the backend should be the default.

        Returns
        -------
        str
        """
        return getattr(openai, 'curr_name', 'openai')

    @classmethod
    def _get_query_func(cls, backend=None):
        """Return current mock function (callable or None)."""
        return cls.name2func[backend or cls.current()]

    @classmethod
    def key(cls):
        """Return current API key. In some cases this is a mock value since
        some modes don't have a key.
        """
        # More reliable than checking name2key because the openai attribute
        # is what's actually used (at least for openai vs. gooseai -
        # huggingface mock_func technically uses a global).
        return openai.api_key

    @classmethod
    def engine(cls, engine_i, backend=None):
        """Get appropriate engine name depending on current api backend and
        selected engine_i.

        Parameters
        ----------
        engine_i: int
            Number from 0-3 (inclusive) specifying which model to use. The two
            backends *should* perform similar for values of 0-2, but openai's
            3 (davinci, 175 billion parameters) is a much bigger model than
            gooseai's 3 (NEO-X, 20 billion parameters). Mostly used in
            query_gpt3().
        backend: str or None
            If provided, should be the name of a backend (e.g. 'huggingface'
            or any of the keys in GPTBackend.backends()).

        Returns
        -------
        str: Name of an engine, e.g. "davinci" if we're in openai mode or
        "gpt-neo-20b" if we're in gooseai mode.
        """
        engines = C.backend_engines[backend or cls.current()]

        # Adds better error message if user passes in a number too big for the
        # current backend.
        try:
            return engines[engine_i]
        except IndexError:
            raise ValueError(f'Encountered invalid engine_i value: {engine_i}.'
                             f'Should be one of {list(range(len(engines)))} '
                             f'when using backend {cls.current()}.')

    # Decorator order matters - doesn't work if we flip these.
    @classmethod
    @with_signature(query_gpt3)
    @add_docstring(query_gpt3)
    def query(cls, prompt, strip_output=True, log=True, **kwargs):
        """

        Parameters
        ----------
        prompt
        strip_output
        log: bool or str
            If True, the logfile defaults to a path like
            './data/logs/2022.04.07.jsonlines' (current year, month, day).
            If str, use that as the log path. If False or None, do not log.
        kwargs

        Returns
        -------
        list[str, dict]
        """
        if not isinstance(prompt, str):
            raise NotImplementedError(
                f'Prompt must be str, not {type(prompt).__name__}.'
            )

        # Keep trunc_full definition here so we can provide warnings if user
        # is in stream mode.
        query_func = cls._get_query_func()
        trunc_full = cls.current() not in cls.skip_trunc
        stream = kwargs.get('stream', False)
        if stream:
            if strip_output:
                warnings.warn('strip_output=True is not supported in stream '
                              'mode. Automatically setting it to False.')
                strip_output = False
            if trunc_full:
                warnings.warn(
                    'Streaming mode does not support manual truncation of '
                    'stop phrases and your current backend has limited '
                    'support for truncation.'
                )

        # V2 library no longer supports user passing in mock_func. We want to
        # remove this from the kwargs we pass to our actual function.
        kwargs_func = kwargs.pop('mock_func', None)
        if kwargs_func:
            raise ValueError(
                f'Encountered unexpected mock_func {kwargs_func} with this '
                'interface. This was part of the v1 library but is no longer '
                'supported.'
            )

        kwargs['prompt'] = prompt
        cls._log_query_kwargs(log=log, query_func=query_func, **kwargs)

        # Possibly easier for caller to check for errors this way? Mostly a
        # holdover from v1 library design, but I'm not 100% sure if the
        # benefits still hold given the new design.
        try:
            text, full_response = query_func(**kwargs)
        except Exception as e:
            raise MockFunctionException(str(e)) from None
        if stream:
            if 'stream' in params(query_func):
                return text, full_response
            # TODO: this isn't yet compatible w/ backends w/ native streaming
            # functionality. Think it should be simple to tweak though since
            # they provide 99% of what I want.
            return stream_multi_response(text, full_response)

        # Manually check for stop phrases because most backends either don't
        # or truncate AFTER the stop phrase which is rarely what we want.
        stop = kwargs.get('stop', [])
        clean_text = []
        # tolist doesn't know how to handle dicts so we check explicitly.
        if not listlike(text):
            text = tolist(text)
            full_response = [full_response]
        for text_, resp_ in zip(text, full_response):
            text_ = truncate_at_first_stop(
                text_,
                stop_phrases=stop,
                finish_reason=resp_.get('finish_reason', ''),
                trunc_full=trunc_full,
                trunc_partial=True
            )
            clean_text.append(strip(text_, strip_output))

        return squeeze(clean_text, full_response, n=kwargs.get('n', 1))

    @classmethod
    def _log_query_kwargs(cls, log, query_func=None, **kwargs):
        """Log kwargs for troubleshooting purposes."""
        if log:
            # Meta key is used to store any info we want to log but that should
            # not be passed to the actual query_gpt3 call.
            kwargs['meta'] = {
                'backend_name': cls.current(),
                'query_func': func_name(query_func) if query_func else None
            }
            if not isinstance(log, (str, Path)):
                log = cls.logger.path
            
            # If log file was deleted, we must recreate it AND use 
            # change_path to reopen the file object.
            with cls.lock:
                if not os.path.exists(log):
                    touch(log)
                    cls.logger.path = None
            if log != cls.logger.path:
                cls.logger.change_path(log)
            cls.logger.info(kwargs)

    def __repr__(self):
        return f'{func_name(self)} <current_name: {self.current()}>'

In [352]:
@with_signature(query_gpt3)
@add_docstring(query_gpt3)
def query_batch(prompt, strip_output=True, log=True, **kwargs):
    kwargs.update(strip_output=strip_output, log=log)
    print('gpt', gpt)
    threads = [
        ReturningThread(target=GPTBackend.query,
                        args=(p,),
                        kwargs=kwargs)
        for p in prompt
    ]
    for thread in threads: thread.start()
    res = [thread.join() for thread in threads]
    if kwargs.get('stream', False):
        return chain(*res)
    return res

In [353]:
prompts = [
    'I am so',
    'Tomorrow is',
    'The color'
]
gpt.switch('repeat')
batch_res = query_batch(prompts, log=False)

Switching openai backend to "repeat".
gpt GPTBackend <current_name: repeat>


In [354]:
batch_res

[('I AM SO', {}), ('TOMORROW IS', {}), ('THE COLOR', {})]

In [355]:
query_batch(prompts, log=True)

gpt GPTBackend <current_name: repeat>
{'prompt': 'I am so', 'meta': {'backend_name': 'repeat', 'query_func': 'query_gpt_repeat'}}
{'prompt': 'Tomorrow is', 'meta': {'backend_name': 'repeat', 'query_func': 'query_gpt_repeat'}}
{'prompt': 'The color', 'meta': {'backend_name': 'repeat', 'query_func': 'query_gpt_repeat'}}


[('I AM SO', {}), ('TOMORROW IS', {}), ('THE COLOR', {})]

In [356]:
!rm {GPTBackend.logger.path}

In [357]:
!cat {GPTBackend.logger.path}

cat: /Users/hmamin/jabberwocky/data/logs/2022.04.08.jsonlines: No such file or directory


In [358]:
query_batch(prompts, n=2)

gpt GPTBackend <current_name: repeat>
{'n': 2, 'prompt': 'Tomorrow is', 'meta': {'backend_name': 'repeat', 'query_func': 'query_gpt_repeat'}}
{'n': 2, 'prompt': 'The color', 'meta': {'backend_name': 'repeat', 'query_func': 'query_gpt_repeat'}}
{'n': 2, 'prompt': 'I am so', 'meta': {'backend_name': 'repeat', 'query_func': 'query_gpt_repeat'}}


[(['I AM SO', 'I AM SO'], [{}, {}]),
 (['TOMORROW IS', 'TOMORROW IS'], [{}, {}]),
 (['THE COLOR', 'THE COLOR'], [{}, {}])]

In [359]:
for tok_, full_ in query_batch(prompts, n=1, stream=True):
    print(tok_, full_)
    if full_['finish_reason']: print()

gpt GPTBackend <current_name: repeat>
{'n': 1, 'stream': True, 'prompt': 'Tomorrow is', 'meta': {'backend_name': 'repeat', 'query_func': 'query_gpt_repeat'}}
{'n': 1, 'stream': True, 'prompt': 'I am so', 'meta': {'backend_name': 'repeat', 'query_func': 'query_gpt_repeat'}}
{'n': 1, 'stream': True, 'prompt': 'The color', 'meta': {'backend_name': 'repeat', 'query_func': 'query_gpt_repeat'}}
I  {'index': 0, 'finish_reason': None}
AM  {'index': 0, 'finish_reason': None}
SO  {'index': 0, 'finish_reason': 'dummy'}

TOMORROW  {'index': 0, 'finish_reason': None}
IS  {'index': 0, 'finish_reason': 'dummy'}

THE  {'index': 0, 'finish_reason': None}
COLOR  {'index': 0, 'finish_reason': 'dummy'}



/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:232: UserWarning: strip_output=True is not supported in stream mode. Automatically setting it to False.
/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:237: UserWarning: Streaming mode does not support manual truncation of stop phrases and your current backend has limited support for truncation.
/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:251: UserWarning: Unused kwargs {'stream': True} received by query_gpt_repeat.
  warnings.warn(f'Unused kwargs {kwargs} received by query_gpt_repeat.')


In [251]:
query_batch(prompts, n=1, stream=False, log=True)

gpt GPTBackend <current_name: repeat>
{'n': 1, 'stream': False, 'prompt': 'I am so', 'meta': {'backend_name': 'repeat', 'query_func': 'query_gpt_repeat'}}
{'n': 1, 'stream': False, 'prompt': 'Tomorrow is', 'meta': {'backend_name': 'repeat', 'query_func': 'query_gpt_repeat'}}
{'n': 1, 'stream': False, 'prompt': 'The color', 'meta': {'backend_name': 'repeat', 'query_func': 'query_gpt_repeat'}}


[('I AM SO', {}), ('TOMORROW IS', {}), ('THE COLOR', {})]

In [262]:
GPTBackend.logger.change_path('./data/logs/2022.04.08.jsonlines')

In [267]:
GPTBackend.logger._log??

In [260]:
vars(GPTBackend.logger.handlers[-1])

{'baseFilename': '/Users/hmamin/jabberwocky/data/logs/2022.04.08.jsonlines',
 'mode': 'a',
 'encoding': None,
 'delay': False,
 'filters': [],
 '_name': None,
 'level': 0,
 'formatter': <jabberwocky.utils.JsonlinesFormatter at 0x11e261dd8>,
 'lock': <unlocked _thread.RLock object owner=0 count=0 at 0x11d62cc30>,
 'stream': <_io.TextIOWrapper name='/Users/hmamin/jabberwocky/data/logs/2022.04.08.jsonlines' mode='a' encoding='UTF-8'>}

In [264]:
vars(GPTBackend.logger.handlers[-1])

{'baseFilename': '/Users/hmamin/jabberwocky/data/logs/2022.04.08.jsonlines',
 'mode': 'a',
 'encoding': None,
 'delay': False,
 'filters': [],
 '_name': None,
 'level': 0,
 'formatter': <jabberwocky.utils.JsonlinesFormatter at 0x11e261dd8>,
 'lock': <unlocked _thread.RLock object owner=0 count=0 at 0x11ec02de0>,
 'stream': <_io.TextIOWrapper name='/Users/hmamin/jabberwocky/data/logs/2022.04.08.jsonlines' mode='a' encoding='UTF-8'>}

In [152]:
with gpt('banana'):
    banana_batch_res = query_batch(prompts, max_tokens=5)

Switching openai backend to "banana".
Switching  backend back to "repeat".


In [153]:
banana_batch_res

[('sorry, my darling,',
  {'id': 'e81fbd5c-5023-4b57-bd7c-897217a54b0f',
   'message': 'success',
   'created': 1649132299,
   'apiVersion': '26 Nov 2021',
   'modelOutputs': [{'output': ' sorry, my darling,', 'input': 'I am so'}]}),
 ('the day when the final',
  {'id': '23f7ac5e-1d4a-4fb0-ba01-ed6f55a47ee3',
   'message': 'success',
   'created': 1649132298,
   'apiVersion': '26 Nov 2021',
   'modelOutputs': [{'output': ' the day when the final',
     'input': 'Tomorrow is'}]}),
 ('is great, it has',
  {'id': '293b870f-deed-44df-b566-5f41543a0894',
   'message': 'success',
   'created': 1649132299,
   'apiVersion': '26 Nov 2021',
   'modelOutputs': [{'output': ' is great, it has', 'input': 'The color'}]})]

In [155]:
# This unzipping business is a bit ugly though. But maybe this can be included
# inside the query_batch function so user never has to do this?
gpt.switch('banana')
for tok_, full_ in query_batch(prompts, max_tokens=5, stream=True):
    print(tok_, select(full_, keep=['index', 'finish_reason']))
    if full_['finish_reason']: print()

Switching openai backend to "banana".


/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:680: UserWarning: query_gpt_banana received unused kwargs {'stream': True}.
  warnings.warn(f'query_gpt_banana received unused kwargs {kwargs}.')


  {'index': 0, 'finish_reason': None}
happy  {'index': 0, 'finish_reason': None}
to  {'index': 0, 'finish_reason': None}
be  {'index': 0, 'finish_reason': None}
back!  {'index': 0, 'finish_reason': 'dummy'}

  {'index': 0, 'finish_reason': None}
the  {'index': 0, 'finish_reason': None}
day  {'index': 0, 'finish_reason': None}
that  {'index': 0, 'finish_reason': None}
I  {'index': 0, 'finish_reason': None}
have  {'index': 0, 'finish_reason': 'dummy'}

ation  {'index': 0, 'finish_reason': None}
of  {'index': 0, 'finish_reason': None}
the  {'index': 0, 'finish_reason': None}
skin  {'index': 0, 'finish_reason': None}
is  {'index': 0, 'finish_reason': 'dummy'}

